# Deep-Learning-from-scratch
## Chap5. Backpropagation
## 5.6 Affine/Softmax 계층 구현하기

### 5.6.1 Affine 계층
신경망의 순전파에서는 가중치 신호의 총합을 계산하기 때문에 행렬곱(np.dot())을 사용했다. <br/>
다음의 예를 통해 다시 한번 복습해보자.

In [1]:
# 앞서 신경망의 순전파를 배울 때 
# 우리는 가중치 신호의 총합을 계산하기 위해 numpy 모듈의 np.dot()를 사용햇다.
# 행렬곱을 사용하여 순전파를 계산해보자

# import numpy library
import numpy as np

X = np.random.rand(2)     # 입력
W = np.random.rand(2, 3)  # 가중치
B = np.random.rand(3)     # 편향

X.shape # (2,)
W.shape # (2, 3)
B.shape # (3,)

Y = np.dot(X, W) + B  # (3,)

In [2]:
print(Y.shape)

(3,)


여기서 X,W,B는 각각 형상이 (2,), (2,3), (3,)인 다차원배열이다. <br/>
뉴런의 가중치 합은 Y = np.dot(X,W) + B처럼 계산한다.<br/>
그리고 이 Y를 활성화 함수로 변환해 다음 층으로 전파하는 것이 신경망 순전파의 흐름이었다.

* Y -> Activation Node -> Next Node

**복습을 조금 더 해보자** <br/>
행렬곱 계산은 대응하는 차원의 원소 수를 일치시키는 게 핵심이다. <br/>
앞선 Y의 출력을 구하는 식에서 X와 W의 곱을 위해서는 대응하는 차원의 원소 수를 일치시켜야 한다. <br/>

In [6]:
print(X.shape)
print(W.shape)

(2,)
(2, 3)


In [3]:
np.dot(X,W)

array([0.66238153, 0.10673713, 0.14458712])

np.dot(W,X) 의 결과는 어떻게 될까?

In [5]:
np.dot(W,X)

ValueError: shapes (2,3) and (2,) not aligned: 3 (dim 1) != 2 (dim 0)

첫 번째 인수 W와 두번째 인수 X의 대응하는 차원의 원소수가 일치하지 않아 Error를 발생시킨다.

* 신경망의 순전파 때 수행하는 행렬 곱은 기하학에서는 어파인 변환(Affine Transformation)라고 한다.
* 이 책에서는 어파인 변환을 수행하는 처리를 'Affine 계층'라는 이름으로 구현한다.

**신경망의 순전파 때 수행하는 행렬곱을 'Affine Transformation'라고 기억해두자!**

In [9]:
 X

array([0.18469147, 0.87256124])

In [10]:
print(np.dot(X, W).shape)

(3,)


### 형상에 주목하며 Affine 계층을 다시 한번 보자!

In [6]:
X = np.random.rand(2)
W = np.random.rand(2, 3)
print('X shape:', X.shape)
print('W shape:', W.shape)

X shape: (2,)
W shape: (2, 3)


np.dot(X, W)

In [7]:
print('np.dot(X,W): ', np.dot(X, W))  
print('np.dot(X,W).shape: ', np.dot(X,W).shape)  # (1, 3), 즉 (3,)의 형상을 뛰게 될 것이다.

np.dot(X,W):  [0.94999045 0.85180734 0.21770842]
np.dot(X,W).shape:  (3,)


### 5.6.2 배치용 Affine 계층

지금까지 설명한 Affine 계층은 입력 데이터로 X 하나만을 고려한 것이었다. <br/>
이번 절에서는 데이터 N개를 묶어 순전파하는 경우, 즉 배치용 Affine 계층을 생각해보자

기존과 다르부분은 입력인 X의 형상이 (N, 2)가 된 것뿐이다.

편향을 더할때도 주의해야 한다.

In [8]:
X_dot_W = np.array([[0, 0, 0],
                    [10, 10, 10]])  # (2, 3)
B = np.array([1, 2, 3])             # (3,)

print('X_dot_W:', X_dot_W)
print('X_dot_W + B: ', X_dot_W + B)

X_dot_W: [[ 0  0  0]
 [10 10 10]]
X_dot_W + B:  [[ 1  2  3]
 [11 12 13]]


순전파의 편향 덧셈은 각각의 데이터에 더해진다.<br/>
그래서 역전파 때는 각 데이터의 역전파 값이 편향의 원소에 모여야 한다. <- ?? 무슨말일까?? <br/>
예를 통해 살펴보자

In [14]:
# 2개의 데이터가 있다고 가정하자

dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [15]:
# 편향의 역전파에서는 
# 앞서 구한 2개의 데이터에 대한 미분을
# 데이터마다 더해서 구한다. 

dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

### Affine 계층 구현 정리하기

In [16]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)  # Affine 계층에서 역전파때 
                                        # 각각의 역전파 값이 편향의 원소에 모여야 한다.
                                        # ???
                                        # 왜 np.sum()을 해주는거지?
                                        # n개의 데이터에 대한 미분을 데이터마다 더해주는 것이다!
        
        return dx


### 5.6.3 Softmax-with-Loss 계층

마지막으로 출력층에서 사용하는 소프트맥스 함수에 관해 설명한다. <br/>
앞에서 말했듯이 소프트맥스 함수는 입력 값을 정규화하여 출력한다.

In [12]:
# 이전 3장 3.5.2에서 배운 것들을 떠올려보자.
# 3.5.2 소프트맥스 함수 구현시 주의할 점

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # 오버플로 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [26]:
# 4.2.4 배치용 크로스엔트로피 오차 구현하기

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [17]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  # 손실
        self.y = None     # softmax의 출력
        self.t = None     # 정답 레이블(원-핫 벡터)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]         # ?
                                             # 정답레이블이 원-핫벡터로 이루어져 있는데
                                             # ??
        dx = (self.y - self.t) / batch_size  # backward에서 dx는 어떻게 구해지는 걸까? 20.02.05.wed pm6:00
        
        return dx

softmax, cross_entropy 앞으로 더욱 자주 볼 것만 같다

'4.2.4 (배치용) 교차 엔트로피 오차 구현하기'를 다시 보자